## This code focuses on visualizing how the CNN behaves in each layer, allowing us to evaluate the behaviour of the AI in recognizing the building images and its typology

#### The GradCAM algorithm were referenced from Selvaraju et al (2016) from his paper in arxiv 

#### "Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization"

#### https://arxiv.org/abs/1610.02391?context=cs.AI

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

from skimage.io import imshow, imread, imsave
import cv2
import os

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.image as mpimg

import time

#### Loading the specified Model

In [ ]:
model = keras.models.load_model('Deep Learning Models\Typology Classifier\Model E - Xception.h5')

In [ ]:
model.summary()

In [ ]:
list_conv = []
for layer in model.layers:
    if (isinstance(layer, keras.layers.Conv2D) or isinstance(layer, keras.layers.SeparableConv2D)):
        k = layer.output_shape[1]
        if(k != 1):
            list_conv.append(layer.name)

In [ ]:
len(list_conv)

In [ ]:
list_conv

In [ ]:
img_size = (256, 256)
preprocess_input = keras.applications.xception.preprocess_input
decode_predictions = keras.applications.xception.decode_predictions

In [ ]:
def get_img_array(img_path, size):
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    array = keras.preprocessing.image.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]

    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy().astype(np.float32)

def save_and_display_gradcam(img_path, heatmap, file_name,layer, cam_path = r"Visualizations/", alpha=0.8, name_index = 1):
    # Load the original image
    img = keras.preprocessing.image.load_img(img_path)
    img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    plt.tight_layout()
    plt.axis('off')
    plt.imshow(superimposed_img)
    plt.title('Layer ' + str(name_index) + ' - ' + layer)
    saving_path = cam_path+file_name+'.jpg'
    plt.savefig(saving_path, transparent=True, bbox_inches='tight')
    plt.show()
    #superimposed_img.save()

    # Display Grad CAM
    #display(Image(saving_path))
    return superimposed_img

In [ ]:
typology_dict = {0 : 'Confined Masonry', 1 : 'RC Infilled Masonry',
                2 : 'Timber Structure', 3 : 'Unconfined Masonry'}

## Mode Display Layer GradCAM

In [ ]:
# The local path to our target image
img_path = r"C:/Users/hafid/OneDrive/Desktop/Foto Validasi/38.jpg"

In [ ]:
# MODE = DISPLAY ALL LAYER

# Prepare image
img_array = get_img_array(img_path, size=img_size)

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array/255.)
print(preds)
print("Predicted:", typology_dict[np.argmax(preds[0])])

# Generate class activation heatmap
for i,layer in enumerate(list_conv):
    heatmap = make_gradcam_heatmap(img_array/255., model, layer)
    if i == len(list_conv)-1:
        img = save_and_display_gradcam(img_path, heatmap, 'Gambar-' + str(i), layer, name_index = 'Final')
    else:
        img = save_and_display_gradcam(img_path, heatmap, 'Gambar-' + str(i), layer, name_index = i+1)
    #time.sleep(0.1)

In [ ]:
# MODE = DISPLAY LAST LAYER
# Prepare image
img_array = get_img_array(img_path, size=img_size)

# Remove last layer's softmax
#model.layers[-1].activation = None

# Print what the top predicted class is
preds = model.predict(img_array)
print(preds)
print("Predicted:", typology_dict[np.argmax(preds[0])])

# Generate class activation heatmap
for i,layer in enumerate(list_conv):
    if i == len(list_conv)-1:
        heatmap = make_gradcam_heatmap(img_array, model, layer)
        if i == len(list_conv)-1:
            img = save_and_display_gradcam(img_path, heatmap, 'Gambar-' + str(i), layer, name_index = 'Final')
        else:
            img = save_and_display_gradcam(img_path, heatmap, 'Gambar-' + str(i), layer, name_index = i+1)

## Miscellaneous validation with structural engineering professionals

In [ ]:
folder_image = "validation_folder_path"

In [ ]:
filenames = []
predictions = []
typologies = []
df = pd.DataFrame(columns=['Filename', 'Prediction_Array','Tipologi'])
for filename in os.listdir(folder_image): 
    image_path = folder_image + "/" + filename
    img_array = get_img_array(image_path, size=img_size)
    preds = model.predict(img_array)
    tipologi = typology_dict[np.argmax(preds[0])]
    print(tipologi)
    filenames.append(filename)
    predictions.append(preds)
    typologies.append(tipologi)
    
df['Filename'] = filenames
df['Prediction_Array'] = predictions
df['Tipologi'] = typologies

In [ ]:
df.to_excel('Tipologi_Validasi.xlsx')